In [ ]:
### evaluation을 위한 평가 코드 (작성용)

import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import os

dist = cv2.DIST_L2

label_dic = {
    "Road": [128, 64, 128],
    "Sidewalk": [232, 35, 244],
    "Building": [70, 70, 70],
    "Wall": [156, 102, 102],
    "Fence": [153, 153, 190],
    "Pole": [153, 153, 153],
    "Traffic Light": [30, 170, 250],
    "Traffic Sign": [0, 220, 220],
    "Vegetation": [35, 142, 107],
    "Terrain": [152, 251, 152],
    "Sky": [180, 130, 70],
    "Parking": [160, 170, 250],
    "Person": [60, 20, 220],
    "Rider": [0, 0, 255],
    "Car": [142, 0, 0],
    "Truck": [70, 0, 0],
    "Bus": [100, 60, 0],
    "Train": [100, 80, 0],
    "Motorcycle": [230, 0, 0],
    "Bicycle": [32, 11, 119]
}

In [ ]:
def wIoU_evaluation(label_dic, gt_img, t_img, weight_map):
    # A simple implementation of wIoU
    # This code is not optimaized
    # You can modify IoU measure codes

    img_H = gt_img.shape[0]
    img_W = gt_img.shape[1]

    iou_stack = []
    for key_idx in label_dic.keys():

        GT = 0 
        TP = 0
        FP = 0 
        IoU = 0

        gt_map   = np.zeros((img_H,img_W,1)) 
        test_map = np.zeros((img_H,img_W,1)) 

        gt_idx   = gt_img == label_dic[key_idx]
        test_idx = t_img  == label_dic[key_idx]

        gt_map[gt_idx[:,:,0]] = 1
        test_map[test_idx[:,:,0]] = 1

        # need to be optimized here
        for i in range(gt_map.shape[0]):
            for j in range(gt_map.shape[1]):

                if gt_map[i,j] == 1:
                    GT += weight_map[i,j]

                if gt_map[i,j] == 1 and test_map[i,j] == 1:
                    TP += weight_map[i,j]

                elif gt_map[i,j] == 0 and test_map[i,j] == 1:
                    FP += weight_map[i,j]

        if GT == 0:
            pass
          

        else:
            IoU = TP/(GT+FP)
            iou_stack.append(IoU)
    
    meaniou = sum(iou_stack)/len(iou_stack)
    
    return meaniou

In [ ]:
def get_all_distance(img, label_dic, dist):
    # distance map generation
    
    dist_map = np.zeros([img.shape[0],img.shape[1]])
    
    
    for key_idx in label_dic.keys():
    
        label = np.abs(img - label_dic[key_idx])
        label_one_ch = label[:,:,0] + label[:,:,1] + label[:,:,2]
       
        target_label = label_one_ch == 0
        target_label = target_label.astype(np.uint8)
        dist_transform = cv2.distanceTransform(target_label, cv2.DIST_L2, 5)
       
        dist_map += dist_transform/(np.max(dist_transform)+0.01)
        
        
    return dist_map

In [ ]:
## evaluate wIOU

data_path = 'dataset'
gt_path = ['d1']
alphas = [0.01,0.1,1,10,100] # boundary importance factor a

for alpha in alphas:
    
    print('-------------')
    print('a:', alpha)

    for gt_idx in gt_path:

        test_path = gt_idx + '_test2'

        # load GT image
        gt_img_path = os.path.join(data_path, gt_idx,'gt.png')
        gt_img = cv2.imread(gt_img_path)

        test_image_paths = os.path.join(data_path,test_path)
        test_img_names = os.listdir(test_image_paths)

        # distance map 
        dist_map = get_all_distance(gt_img, label_dic, dist)
        weight_map = np.exp(-alpha*dist_map)


        for img_name in test_img_names:

            test_img_path = os.path.join(test_image_paths,img_name)
            t_img  = cv2.imread(test_img_path)

            IoU = wIoU_evaluation(label_dic, gt_img, t_img, weight_map)

            print(IoU)             
   
print('done')